In [137]:
import yfinance as yf
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Descargar datos de acciones
ticker = "BTC-USD"  # Cambia esto por el símbolo de la acción que desees
data = yf.download(ticker, start="2023-01-01", end="2023-07-01")

# Comprobar los datos descargados
print(data.head())

# Determinar los colores de las barras de volumen
colors = ['green' if close > open else 'red' if close < open else 'gray' 
          for open, close in zip(data['Open'], data['Close'])]

# Crear el gráfico con subplots
fig = make_subplots(
    rows=2, cols=1, shared_xaxes=True, 
    vertical_spacing=0.02, 
    row_heights=[0.7, 0.3],  # Ajustar la altura relativa de los subplots
    subplot_titles=(f"Gráfico de Velas Japonesas para {ticker}", "Volumen de Transacción")
)

# Añadir velas japonesas
fig.add_trace(go.Candlestick(
    x=data.index,
    open=data['Open'],
    high=data['High'],
    low=data['Low'],
    close=data['Close'],
    increasing_line_color='green', 
    decreasing_line_color='red',
    showlegend=False
), row=1, col=1)

# Añadir gráfico de barras para el volumen con colores condicionales
fig.add_trace(go.Bar(
    x=data.index,
    y=data['Volume'],
    marker_color=colors,  # Asignar los colores condicionales
    showlegend=False
), row=2, col=1)

# Configurar el diseño del gráfico
fig.update_layout(
    title_text=f'Análisis de {ticker} - Velas Japonesas y Volumen',
    xaxis_title='Fecha',
    yaxis_title='Precio ($)',
    xaxis_rangeslider_visible=False,  # Ocultar el control deslizante de rango
    template='plotly_white',  # Aplicar el tema plotly_white
    font=dict(family="Arial", size=12),  # Establecer fuente del texto
    xaxis=dict(title=dict(text='Fecha', standoff=15)),  # Ajustar el espacio entre el título del eje X y las etiquetas
    xaxis2=dict(title=dict(text='Fecha', standoff=15))  # Asegurar que el título del eje X del subplot inferior también esté ajustado
)

# Mostrar el gráfico
fig.show()


[*********************100%%**********************]  1 of 1 completed

                    Open          High           Low         Close  \
Date                                                                 
2023-01-01  16547.914062  16630.439453  16521.234375  16625.080078   
2023-01-02  16625.509766  16759.343750  16572.228516  16688.470703   
2023-01-03  16688.847656  16760.447266  16622.371094  16679.857422   
2023-01-04  16680.205078  16964.585938  16667.763672  16863.238281   
2023-01-05  16863.472656  16884.021484  16790.283203  16836.736328   

               Adj Close       Volume  
Date                                   
2023-01-01  16625.080078   9244361700  
2023-01-02  16688.470703  12097775227  
2023-01-03  16679.857422  13903079207  
2023-01-04  16863.238281  18421743322  
2023-01-05  16836.736328  13692758566  


In [138]:
import yfinance as yf
import numpy as np
import pandas as pd
import plotly.graph_objs as go
from datetime import datetime
import yfinance as yf
import matplotlib.pyplot as plt

ahora = datetime.now().strftime('%Y-%m-%d')

Ticket = 'BTC-USD'

# Descargar los datos de QQQ
data = yf.download(Ticket, start="2023-01-01", end=ahora,interval='1d')

# Obtener los rendimientos diarios
returns = data['Adj Close'].pct_change().dropna()

# Configurar las simulaciones
n_simulations = 10
n_days = len(returns)

# Realizar las simulaciones de bootstrap
bootstrap_samples = np.random.choice(returns, (n_simulations, n_days), replace=True)
bootstrap_cumulated = np.cumprod(1 + bootstrap_samples, axis=1)

# Calcular el precio simulado para cada muestra de bootstrap
simulated_prices = bootstrap_cumulated * data['Adj Close'].iloc[0]

# Crear un DataFrame para las simulaciones
simulated_df = pd.DataFrame(simulated_prices.T)
simulated_df.set_index(data.index[:-1],inplace=True)

# Graficar las simulaciones
fig = go.Figure()

# Agregar líneas para cada simulación
for i in range(n_simulations):
    fig.add_trace(go.Scatter(x=data.index, y=simulated_df[i], mode='lines', line=dict(width=2), opacity=0.3))

# Agregar línea del precio real
fig.add_trace(go.Scatter(x=data.index, y=data['Adj Close'], mode='lines', name='Real', line=dict(color='black', width=2)))

# Configurar el gráfico
fig.update_layout(title=f'Simulaciones de Bootstrap para {Ticket}',
                  xaxis_title='Fecha',
                  yaxis_title='Precio Ajustado',
                  template='plotly_white')

# Mostrar el gráfico
fig.show()


[*********************100%%**********************]  1 of 1 completed


In [139]:
import yfinance as yf
import numpy as np
import plotly.graph_objects as go


Ticket = 'BTC-USD'

# Descargar los datos de QQQ
data = yf.download(Ticket, start="2024-01-01", end=ahora,interval='1d')

# Calcular los retornos diarios
data['Return'] = data['Close'].pct_change()

# Establecer un umbral para detectar eventos (e.g., 1% de cambio)
threshold = 0.01

eventos_si_no = data['Return'].abs() > threshold

# Crear una serie de eventos basada en el umbral
events = data.index[eventos_si_no]

# Crear una figura de Plotly
fig = go.Figure()

# Agregar la línea del precio de cierre
fig.add_trace(go.Scatter(
    x=data.index, 
    y=data['Close'], 
    mode='lines', 
    name='Close Price',
    line=dict(color='blue')
))

# Agregar los eventos como puntos
fig.add_trace(go.Scatter(
    x=events, 
    y=data.loc[events, 'Close'], 
    mode='markers', 
    name='Events',
    marker=dict(color='red', size=8)
))

# Personalizar el diseño
fig.update_layout(
    title='Precio de Cierre de AAPL con Eventos Detectados',
    xaxis_title='Fecha',
    yaxis_title='Precio de Cierre (USD)',
    legend_title='Leyenda',
    template='plotly_white',
    xaxis=dict(
        showgrid=True,
        gridcolor='lightgrey'
    ),
    yaxis=dict(
        showgrid=True,
        gridcolor='lightgrey'
    )
)

# Mostrar la figura
fig.show()


[*********************100%%**********************]  1 of 1 completed


In [140]:
import numpy as np
import plotly.graph_objs as go
import pandas as pd  # Importamos pandas para manejar las fechas

def funcion_intensidad(mu, alpha, beta, eventos, tiempos):
    # Calcula la intensidad actual del proceso de Hawkes
    return mu + alpha * np.sum(eventos * np.exp(-beta * (len(eventos) + 1 - tiempos)))

def simulated_hawkes_closing_prices(eventos, lambda_0, alpha, beta, initial_price, daily_volatility, start_date, num_days=30):
    number_events = len(eventos)
    
    eventos_historicos = list(eventos)  # Convertir a lista para modificarla

    # Crear un array de tiempos para los eventos históricos
    historical_times = np.arange(1, number_events + 1)

    # Lista para almacenar precios de cierre
    prices = [initial_price]

    current_intensity = lambda_0  # Intensidad actual
    
    simulated_events = np.zeros(1)
    
    for i in range(1, num_days + 1):
        # Actualiza los tiempos históricos añadiendo el nuevo tiempo
        historical_times = np.append(historical_times, number_events + i)

        P = 1 - np.exp(-current_intensity)
        
        price_change = 0  # Inicializar el cambio de precio para el día
        
        if np.random.uniform() < P:
            eventos_historicos.append(1)
            # Calcula el cambio de precio basado en el número de eventos
            event_price_change = np.random.normal(loc=0, scale=daily_volatility)
            price_change += event_price_change
            simulated_events = np.append(simulated_events, 1)
        else:
            simulated_events = np.append(simulated_events, 0)
            eventos_historicos.append(0)

        # Actualiza la intensidad actual del proceso de Hawkes
        current_intensity = funcion_intensidad(lambda_0, alpha, beta, eventos_historicos, historical_times)
        
        price_change += np.random.normal(loc=0, scale=daily_volatility)
        new_price = prices[-1] + price_change
        prices.append(new_price)

    # Generar las fechas desde la fecha de inicio
    dates = pd.date_range(start=start_date, periods=num_days + 1)  # +1 para incluir el día inicial

    return dates, prices, simulated_events

# Parámetros del modelo
lambda_0 = 0.5  # Intensidad base
alpha = 0.8     # Impacto del evento
beta = 1.5      # Tasa de decaimiento
initial_price = data['Close'][-1]  # Precio inicial
daily_volatility = 1  # Volatilidad diaria
num_days = 140  # Número de días
start_date = '2024-08-06'  # Fecha de inicio

# Inicializar la lista de eventos
events = eventos_si_no#[0, 1, 0, 0, 1, 0, 1, 0]  # Ejemplo de inicialización, reemplaza con tu 'eventos_si_no'

# Simulación de múltiples trayectorias
num_simulations = 5  # Número de simulaciones que quieres realizar
all_simulations = []  # Almacena todas las simulaciones

for _ in range(num_simulations):
    dates, prices, simulated_events = simulated_hawkes_closing_prices(events, lambda_0, alpha, beta, initial_price, daily_volatility, start_date, num_days)
    all_simulations.append((dates, prices, simulated_events))

# Crear gráfico con Plotly
fig = go.Figure()

# Añadir cada simulación como una línea separada
for idx, (dates, prices, simulated_events) in enumerate(all_simulations):
    # Añadir la línea de precios de cierre para cada simulación
    fig.add_trace(go.Scatter(
        x=dates,
        y=prices[1:],  # Saltar el precio inicial para ajustar la longitud de la serie temporal
        mode='lines',
        name=f'Simulación {idx + 1}',
        line=dict(width=1)  # Cambiar el ancho de línea si lo deseas
    ))

    # Identificar los tiempos de eventos (donde los eventos son 1)
    event_dates = [dates[i-1] for i, e in enumerate(simulated_events) if e == 1 and i < num_days]

    # Añadir el scatter plot para eventos de cada simulación
    fig.add_trace(go.Scatter(
        x=event_dates,
        y=[prices[i] for i, e in enumerate(simulated_events) if e == 1 and i < num_days],
        mode='markers',
        #name=f'Eventos Simulación {idx + 1}',
        marker=dict(color='red', symbol='x', size=5)
    ))

# Actualizar el diseño del gráfico
fig.update_layout(
    title='Simulación de Precios de Cierre usando Proceso de Hawkes',
    xaxis_title='Fechas',
    yaxis_title='Precio',
    #legend=dict(x=0, y=1),
    template='plotly_white'
)

# Mostrar el gráfico
fig.show()


/var/folders/0q/l5j608j10jq82grhg4m4pf0h0000gn/T/ipykernel_62401/3974348226.py:58: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



In [146]:
import yfinance as yf
import numpy as np
import plotly.graph_objects as go
import pandas as pd

# Cargar datos
#data = yf.download('AAPL', start='2023-01-01', end='2024-01-01')

# Calcular los retornos diarios
data['Return'] = data['Close'].pct_change()

# Establecer un umbral para detectar eventos (e.g., 1% de cambio)
threshold = 0.01

eventos_si_no = data['Return'].abs() > threshold

# Crear una serie de eventos basada en el umbral
events = data.index[eventos_si_no]

# Función para calcular la intensidad del proceso de Hawkes
def funcion_intensidad(mu, alpha, beta, eventos, tiempos):
    # Calcula la intensidad actual del proceso de Hawkes
    return mu + alpha * np.sum(eventos * np.exp(-beta * (len(eventos) + 1 - tiempos)))

# Simulación de precios de cierre usando el proceso de Hawkes
def simulated_hawkes_closing_prices(eventos, lambda_0, alpha, beta, initial_price, daily_volatility, start_date, num_days=30):
    number_events = len(eventos)
    
    eventos_historicos = list(eventos)  # Convertir a lista para modificarla

    # Crear un array de tiempos para los eventos históricos
    historical_times = np.arange(1, number_events + 1)

    # Lista para almacenar precios de cierre
    prices = [initial_price]

    current_intensity = lambda_0  # Intensidad actual
    
    simulated_events = np.zeros(1)
    
    for i in range(1, num_days + 1):
        # Actualiza los tiempos históricos añadiendo el nuevo tiempo
        historical_times = np.append(historical_times, number_events + i)

        P = 1 - np.exp(-current_intensity)
        
        price_change = 0  # Inicializar el cambio de precio para el día
        
        if np.random.uniform() < P:
            eventos_historicos.append(1)
            # Calcula el cambio de precio basado en el número de eventos
            event_price_change = np.random.normal(loc=0, scale=daily_volatility)
            price_change += event_price_change
            simulated_events = np.append(simulated_events, 1)
        else:
            simulated_events = np.append(simulated_events, 0)
            eventos_historicos.append(0)

        # Actualiza la intensidad actual del proceso de Hawkes
        current_intensity = funcion_intensidad(lambda_0, alpha, beta, eventos_historicos, historical_times)
        
        price_change += np.random.normal(loc=0, scale=daily_volatility)
        new_price = prices[-1] + price_change
        prices.append(new_price)

    # Generar las fechas desde la fecha de inicio
    dates = pd.date_range(start=start_date, periods=num_days + 1)  # +1 para incluir el día inicial

    return dates, prices, simulated_events

# Parámetros del modelo
lambda_0 = 0.5  # Intensidad base
alpha = 0.8     # Impacto del evento
beta = 1.5      # Tasa de decaimiento
initial_price = data['Close'][-1]  # Precio inicial
daily_volatility = 1000  # Volatilidad diaria
num_days = 140  # Número de días
start_date = ahora  # Fecha de inicio

# Inicializar la lista de eventos
events = eventos_si_no

# Simulación de múltiples trayectorias
num_simulations = 5  # Número de simulaciones que quieres realizar
all_simulations = []  # Almacena todas las simulaciones

for _ in range(num_simulations):
    dates, prices, simulated_events = simulated_hawkes_closing_prices(events, lambda_0, alpha, beta, initial_price, daily_volatility, start_date, num_days)
    all_simulations.append((dates, prices, simulated_events))

# Crear un gráfico combinado de Plotly
combined_fig = go.Figure()

# Añadir la línea del precio de cierre real
combined_fig.add_trace(go.Scatter(
    x=data.index, 
    y=data['Close'], 
    mode='lines', 
    name='Close Price',
    line=dict(color='black')
))

# Añadir los eventos detectados como puntos en el gráfico real
combined_fig.add_trace(go.Scatter(
    x=events[events].index,
    y=data.loc[events, 'Close'], 
    mode='markers', 
    name='Events',
    marker=dict(color='red', size=4)
))
print(events)
# Añadir cada simulación como una línea separada
for idx, (dates, prices, simulated_events) in enumerate(all_simulations):
    # Añadir la línea de precios de cierre para cada simulación
    combined_fig.add_trace(go.Scatter(
        x=dates,
        y=prices[1:],  # Saltar el precio inicial para ajustar la longitud de la serie temporal
        mode='lines',
        name=f'Simulación {idx + 1}',
        line=dict(width=2)  # Cambiar el ancho de línea si lo deseas
    ))

    # Identificar los tiempos de eventos (donde los eventos son 1)
    event_dates = [dates[i-1] for i, e in enumerate(simulated_events) if e == 1 and i < num_days]

    # Añadir el scatter plot para eventos de cada simulación
    combined_fig.add_trace(go.Scatter(
        x=event_dates,
        y=[prices[i] for i, e in enumerate(simulated_events) if e == 1 and i < num_days],
        mode='markers',
        marker=dict(color='gray', symbol='0', size=4),
        showlegend=False
    ))

# Actualizar el diseño del gráfico combinado
combined_fig.update_layout(
    title='Comparación del Precio de Cierre Real con Simulaciones usando Proceso de Hawkes',
    xaxis_title='Fecha',
    yaxis_title='Precio de Cierre (USD)',
    legend_title='Leyenda',
    template='plotly_white',
    xaxis=dict(
        showgrid=True,
        gridcolor='lightgrey'
    ),
    yaxis=dict(
        showgrid=True,
        gridcolor='lightgrey'
    )
)

# Mostrar el gráfico combinado
combined_fig.show()


Date
2024-01-01    False
2024-01-02     True
2024-01-03     True
2024-01-04     True
2024-01-05    False
              ...  
2024-08-02     True
2024-08-03     True
2024-08-04     True
2024-08-05     True
2024-08-06     True
Name: Return, Length: 219, dtype: bool


/var/folders/0q/l5j608j10jq82grhg4m4pf0h0000gn/T/ipykernel_62401/2260162546.py:75: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



In [148]:
import numpy as np
import pandas as pd
import plotly.graph_objs as go
import plotly.subplots as sp
import yfinance as yf
from datetime import datetime

ahora = datetime.now().strftime('%Y-%m-%d')

Ticket = 'BTC-USD'

# Descargar los datos de QQQ
data = yf.download(Ticket, start="2024-01-01", end=ahora,interval='1d')

ahora = datetime.now().strftime('%Y-%m-%d')

Ticket = 'BTC-USD'

# Descargar los datos de QQQ
data = yf.download(Ticket, start="2023-11-01", end=ahora,interval='1d')

data = data['Close']

# Funciones de cálculo de indicadores

# 1. Media Móvil Simple (SMA)
def simple_moving_average(data, period):
    return data.rolling(window=period).mean()

# 2. Media Móvil Exponencial (EMA)
def exponential_moving_average(data, period):
    return data.ewm(span=period, adjust=False).mean()

# 3. Relative Strength Index (RSI)
def relative_strength_index(data, period):
    delta = data.diff(1)
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    
    avg_gain = gain.rolling(window=period).mean()
    avg_loss = loss.rolling(window=period).mean()
    
    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    
    return rsi

# 4. Moving Average Convergence Divergence (MACD)
def macd(data, short_period=12, long_period=26, signal_period=9):
    short_ema = data.ewm(span=short_period, adjust=False).mean()
    long_ema = data.ewm(span=long_period, adjust=False).mean()
    
    macd_line = short_ema - long_ema
    signal_line = macd_line.ewm(span=signal_period, adjust=False).mean()
    macd_histogram = macd_line - signal_line
    
    return macd_line, signal_line, macd_histogram

# 5. Tasa de Cambio (ROC)
def rate_of_change(data, period):
    return ((data - data.shift(period)) / data.shift(period)) * 100

# 6. Índice de Canal de Commodities (CCI)
def commodity_channel_index(data, period):
    sma = data.rolling(window=period).mean()
    deviation = data - sma
    mean_deviation = deviation.abs().rolling(window=period).mean()
    
    cci = (data - sma) / (0.015 * mean_deviation)
    return cci

# 7. Momentum
def momentum(data, period):
    return data.diff(period)

# 8. Media Móvil Ponderada (WMA)
def weighted_moving_average(data, period):
    weights = np.arange(1, period + 1)
    wma = data.rolling(window=period).apply(lambda prices: np.dot(prices, weights) / weights.sum(), raw=True)
    return wma

# 9. Desviación Estándar
def standard_deviation(data, period):
    return data.rolling(window=period).std()

# Calcular los indicadores
sma = simple_moving_average(data, 14)
ema = exponential_moving_average(data, 14)
rsi = relative_strength_index(data, 14)
macd_line, signal_line, macd_histogram = macd(data)
roc = rate_of_change(data, 14)
cci = commodity_channel_index(data, 14)
mom = momentum(data, 14)
wma = weighted_moving_average(data, 14)
std_dev = standard_deviation(data, 14)

# Crear gráficos con Plotly
fig = sp.make_subplots(rows=3, cols=2, shared_xaxes=True, vertical_spacing=0.05,
                       subplot_titles=('','Precio de Cierre y Medias Móviles', 'RSI', 'MACD', 'Momentum, ROC','CCI'))

# Gráfico de Precios de Cierre y Medias Móviles
fig.add_trace(go.Scatter(x=data.index, y=data, mode='lines', name='Precio de Cierre', line=dict(color='grey')), row=1, col=2)
fig.add_trace(go.Scatter(x=data.index, y=sma, mode='lines', name='SMA (14)', line=dict(color='blue')), row=1, col=2)
fig.add_trace(go.Scatter(x=data.index, y=ema, mode='lines', name='EMA (14)', line=dict(color='green')), row=1, col=2)

# Gráfico de RSI
fig.add_trace(go.Scatter(x=data.index, y=rsi, mode='lines', name='RSI (14)', line=dict(color='purple')), row=2, col=1)
fig.add_shape(type='line', x0=data.index[0], x1=data.index[-1], y0=70, y1=70, line=dict(color='red', dash='dash'), row=2, col=1)
fig.add_shape(type='line', x0=data.index[0], x1=data.index[-1], y0=30, y1=30, line=dict(color='green', dash='dash'), row=2, col=1)

# Gráfico de MACD
fig.add_trace(go.Scatter(x=data.index, y=macd_line, mode='lines', name='Línea MACD', line=dict(color='blue')), row=2, col=2)
fig.add_trace(go.Scatter(x=data.index, y=signal_line, mode='lines', name='Línea de Señal', line=dict(color='orange')), row=2, col=2)
fig.add_trace(go.Bar(x=data.index, y=macd_histogram, name='Histograma MACD', marker_color='grey'), row=2, col=2)

# Gráfico de Momentum, ROC, CCI
fig.add_trace(go.Scatter(x=data.index, y=mom, mode='lines', name='Momentum (14)', line=dict(color='lightgreen')), row=3, col=1)
fig.add_trace(go.Scatter(x=data.index, y=roc, mode='lines', name='ROC (14)', line=dict(color='magenta')), row=3, col=1)
fig.add_trace(go.Scatter(x=data.index, y=cci, mode='lines', name='CCI (14)', line=dict(color='brown')), row=3, col=2)

# Layout del gráfico
fig.update_layout(title='Indicadores Técnicos',
                  xaxis_title='Fecha',
                  yaxis_title='Valor',
                  height=800,
                  showlegend=True,
                  legend=dict(x=0, y=1.0))

fig.update_xaxes(rangeslider_visible=False)
fig.show()


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [149]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from scipy.stats import shapiro, probplot
from statsmodels.stats.diagnostic import het_breuschpagan, acorr_breusch_godfrey
from statsmodels.tsa.stattools import adfuller, acf, pacf

def analyze_residuals(prices: pd.Series):
    # Calcular retornos logarítmicos
    returns = np.log(prices / prices.shift(1)).dropna()

    # Crear un DataFrame con los retornos
    data = pd.DataFrame({'Returns': returns})

    # Ajustar un modelo de regresión lineal simple
    data['Time'] = np.arange(len(data))
    X = sm.add_constant(data['Time'])
    model = sm.OLS(data['Returns'], X).fit()
    data['Fitted'] = model.fittedvalues
    data['Residuals'] = model.resid

    # Mostrar resumen del modelo
    print("Resumen del Modelo:")
    print(model.summary())

    # Análisis de residuos
    residuals = data['Residuals']

    # 1. Normalidad de los residuos
    hist_fig = go.Histogram(x=residuals, nbinsx=20, name='Residuals', marker=dict(color='rgba(0, 123, 255, 0.7)'))

    # Prueba de normalidad de Shapiro-Wilk
    shapiro_stat, shapiro_p_value = shapiro(residuals)
    print(f"Prueba de Shapiro-Wilk: Estadístico = {shapiro_stat:.4f}, p-valor = {shapiro_p_value:.4f}")
    if shapiro_p_value > 0.05:
        print("Interpretación: Los residuos siguen una distribución normal (p-valor > 0.05).")
    else:
        print("Interpretación: Los residuos no siguen una distribución normal (p-valor <= 0.05).")

    # Gráfico Q-Q
    qq = probplot(residuals, dist="norm")
    qq_fig = go.Scatter(x=qq[0][0], y=qq[0][1], mode='markers', name='Datos')
    qq_line = go.Scatter(x=qq[0][0], y=qq[0][0], mode='lines', name='Línea de referencia')

    # 2. Homocedasticidad
    scatter_fig = go.Scatter(x=data['Fitted'], y=residuals, mode='markers', name='Residuos')
    scatter_line = go.Scatter(x=[data['Fitted'].min(), data['Fitted'].max()], y=[0, 0], mode='lines', name='Línea 0', line=dict(color='red', dash='dash'))

    # Prueba de heterocedasticidad de Breusch-Pagan
    bp_test = het_breuschpagan(residuals, X)
    print(f"Prueba de Breusch-Pagan: Estadístico = {bp_test[0]:.4f}, p-valor = {bp_test[1]:.4f}")
    if bp_test[1] > 0.05:
        print("Interpretación: No hay evidencia de heterocedasticidad (p-valor > 0.05).")
    else:
        print("Interpretación: Hay evidencia de heterocedasticidad (p-valor <= 0.05).")

    # 3. Independencia y autocorrelación de residuos
    line_fig = go.Scatter(x=data.index, y=residuals, mode='lines', name='Serie Temporal de Residuos')

    # Correlograma de los residuos
    acf_values = acf(residuals, nlags=20)
    pacf_values = pacf(residuals, nlags=20)

    acf_fig = go.Bar(x=np.arange(len(acf_values)), y=acf_values, name='ACF', marker=dict(color='rgba(255, 99, 71, 0.7)'))
    pacf_fig = go.Bar(x=np.arange(len(pacf_values)), y=pacf_values, name='PACF', marker=dict(color='rgba(71, 99, 255, 0.7)'))

    # Prueba de autocorrelación de Breusch-Godfrey
    bg_test = acorr_breusch_godfrey(model, nlags=5)
    print(f"Prueba de Breusch-Godfrey: Estadístico = {bg_test[0]:.4f}, p-valor = {bg_test[1]:.4f}")
    if bg_test[1] > 0.05:
        print("Interpretación: No hay evidencia de autocorrelación (p-valor > 0.05).")
    else:
        print("Interpretación: Hay evidencia de autocorrelación (p-valor <= 0.05).")

    # ADF Test para verificar si la serie de residuos es estacionaria
    adf_stat, adf_p_value, _, _, _, _ = adfuller(residuals)
    print(f"Prueba de ADF (Dickey-Fuller): Estadístico = {adf_stat:.4f}, p-valor = {adf_p_value:.4f}")
    if adf_p_value <= 0.05:
        print("Interpretación: Los residuos son estacionarios (p-valor <= 0.05).")
    else:
        print("Interpretación: Los residuos no son estacionarios (p-valor > 0.05).")

    # Crear subplots
    fig = make_subplots(rows=3, cols=2, 
                        subplot_titles=("Histograma de Residuos", "Gráfico Q-Q de Residuos",
                                        "Residuos vs. Valores Ajustados", "Serie Temporal de Residuos",
                                        "Correlograma ACF", "Correlograma PACF"))

    # Añadir gráficos
    fig.add_trace(hist_fig, row=1, col=1)
    fig.add_trace(qq_fig, row=1, col=2)
    fig.add_trace(qq_line, row=1, col=2)
    fig.add_trace(scatter_fig, row=2, col=1)
    fig.add_trace(scatter_line, row=2, col=1)
    fig.add_trace(line_fig, row=2, col=2)
    fig.add_trace(acf_fig, row=3, col=1)
    fig.add_trace(pacf_fig, row=3, col=2)

    # Actualizar layout
    fig.update_layout(title='Análisis de Residuos', template='plotly_white', showlegend=False, height=900)

    # Mostrar figura
    fig.show()

# Ejemplo de uso con una serie de precios ficticia
fechas = pd.date_range(start='2024-01-01', periods=100)
precios = pd.Series(np.cumsum(np.random.normal(0, 1, 100)) + 100, index=fechas)

analyze_residuals(returns)


Resumen del Modelo:
                            OLS Regression Results                            
Dep. Variable:                Returns   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.003
Method:                 Least Squares   F-statistic:                   0.08036
Date:                Wed, 07 Aug 2024   Prob (F-statistic):              0.777
Time:                        15:57:12   Log-Likelihood:                -554.20
No. Observations:                 270   AIC:                             1112.
Df Residuals:                     268   BIC:                             1120.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.2074      0.230

/Users/luis/Desktop/UNIVERSIDAD/Simulación/Hawkes/HawkesEnv/lib/python3.12/site-packages/pandas/core/arraylike.py:399: RuntimeWarning:

invalid value encountered in log



In [135]:
import yfinance as yf
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.subplots as sp

# Descargar datos históricos de Apple
ticker = 'AAPL'
data = yf.download(ticker, start='2023-01-01', end='2024-01-01')

# Descargar datos históricos del VIX
vix_data = yf.download('^VIX', start='2023-01-01', end='2024-01-01')

# Calcular rendimientos logarítmicos
data['Log Returns'] = np.log(data['Close'] / data['Close'].shift(1))

# Calcular Volatilidad Histórica
data['Historical Volatility'] = data['Log Returns'].rolling(window=21).std() * np.sqrt(252)

# Calcular Bollinger Bands
data['Middle Band'] = data['Close'].rolling(window=20).mean()
data['Upper Band'] = data['Middle Band'] + 2 * data['Close'].rolling(window=20).std()
data['Lower Band'] = data['Middle Band'] - 2 * data['Close'].rolling(window=20).std()

# Calcular ATR (Average True Range)
data['High-Low'] = data['High'] - data['Low']
data['High-Close'] = np.abs(data['High'] - data['Close'].shift(1))
data['Low-Close'] = np.abs(data['Low'] - data['Close'].shift(1))
data['True Range'] = data[['High-Low', 'High-Close', 'Low-Close']].max(axis=1)
data['ATR'] = data['True Range'].rolling(window=14).mean()

# Calcular Chaikin Volatility
data['EMA High'] = data['High'].ewm(span=10, adjust=False).mean()
data['EMA Low'] = data['Low'].ewm(span=10, adjust=False).mean()
data['Chaikin Volatility'] = ((data['EMA High'] - data['EMA Low']) / data['EMA Low']).rolling(window=10).mean()

# Calcular Keltner Channels
data['EMA Close'] = data['Close'].ewm(span=20, adjust=False).mean()
data['Upper Keltner'] = data['EMA Close'] + data['ATR'] * 2
data['Lower Keltner'] = data['EMA Close'] - data['ATR'] * 2

# Crear subplots para los gráficos
fig = sp.make_subplots(rows=3, cols=2, subplot_titles=(
    'Volatilidad Histórica',
    'Bollinger Bands',
    'Average True Range (ATR)',
    'Chaikin Volatility',
    'Keltner Channels',
    'VIX (CBOE Volatility Index)'
))

# Volatilidad Histórica
fig.add_trace(
    go.Scatter(x=data.index, y=data['Historical Volatility'], mode='lines', name='Volatilidad Histórica', line=dict(color='blue')),
    row=1, col=1
)

# Bollinger Bands
fig.add_trace(
    go.Scatter(x=data.index, y=data['Close'], mode='lines', name='Precio de Cierre', line=dict(color='black')),
    row=1, col=2
)
fig.add_trace(
    go.Scatter(x=data.index, y=data['Upper Band'], mode='lines', name='Banda Superior', line=dict(color='red', dash='dash')),
    row=1, col=2
)
fig.add_trace(
    go.Scatter(x=data.index, y=data['Middle Band'], mode='lines', name='Banda Media', line=dict(color='blue', dash='dash')),
    row=1, col=2
)
fig.add_trace(
    go.Scatter(x=data.index, y=data['Lower Band'], mode='lines', name='Banda Inferior', line=dict(color='green', dash='dash')),
    row=1, col=2
)

# ATR
fig.add_trace(
    go.Scatter(x=data.index, y=data['ATR'], mode='lines', name='ATR', line=dict(color='purple')),
    row=2, col=1
)

# Chaikin Volatility
fig.add_trace(
    go.Scatter(x=data.index, y=data['Chaikin Volatility'], mode='lines', name='Chaikin Volatility', line=dict(color='orange')),
    row=2, col=2
)

# Keltner Channels
fig.add_trace(
    go.Scatter(x=data.index, y=data['Close'], mode='lines', name='Precio de Cierre', line=dict(color='black')),
    row=3, col=1
)
fig.add_trace(
    go.Scatter(x=data.index, y=data['Upper Keltner'], mode='lines', name='Keltner Superior', line=dict(color='red', dash='dash')),
    row=3, col=1
)
fig.add_trace(
    go.Scatter(x=data.index, y=data['EMA Close'], mode='lines', name='EMA', line=dict(color='blue', dash='dash')),
    row=3, col=1
)
fig.add_trace(
    go.Scatter(x=data.index, y=data['Lower Keltner'], mode='lines', name='Keltner Inferior', line=dict(color='green', dash='dash')),
    row=3, col=1
)

# VIX
fig.add_trace(
    go.Scatter(x=vix_data.index, y=vix_data['Close'], mode='lines', name='VIX', line=dict(color='magenta')),
    row=3, col=2
)

# Layout
fig.update_layout(
    height=900,
    width=1200,
    title_text='Análisis de Volatilidad y Bandas de Precios para AAPL',
    showlegend=True
)

fig.update_xaxes(title_text='Fecha', row=1, col=1)
fig.update_xaxes(title_text='Fecha', row=1, col=2)
fig.update_xaxes(title_text='Fecha', row=2, col=1)
fig.update_xaxes(title_text='Fecha', row=2, col=2)
fig.update_xaxes(title_text='Fecha', row=3, col=1)
fig.update_xaxes(title_text='Fecha', row=3, col=2)

fig.update_yaxes(title_text='Volatilidad', row=1, col=1)
fig.update_yaxes(title_text='Precio', row=1, col=2)
fig.update_yaxes(title_text='ATR', row=2, col=1)
fig.update_yaxes(title_text='Volatilidad', row=2, col=2)
fig.update_yaxes(title_text='Precio', row=3, col=1)
fig.update_yaxes(title_text='VIX', row=3, col=2)

fig.show()


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
